In [1]:
#importing libraries
import pandas as pd
import numpy as np
import os
from datetime import datetime

In [2]:
raw_confirmed=pd.read_csv('confirmed.csv')
raw_deaths=pd.read_csv('deaths.csv')
raw_recovered=pd.read_csv('recovered.csv')

In [3]:
raw_recovered.shape

(225, 50)

In [4]:
# Un-pivoting the data

raw_data_confirmed = pd.melt(raw_confirmed, id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], var_name=['Date'])
raw_data_deaths = pd.melt(raw_deaths, id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], var_name=['Date'])
raw_data_Recovered = pd.melt(raw_recovered, id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], var_name=['Date'])

In [5]:
raw_data_confirmed.shape

(10350, 6)

In [6]:
raw_data_confirmed.head()

,Province/State,Country/Region,Lat,Long,Date,value
0,Anhui,Mainland China,31.8257,117.2264,1/22/20,1
1,Beijing,Mainland China,40.1824,116.4142,1/22/20,14
2,Chongqing,Mainland China,30.0572,107.8740,1/22/20,6
3,Fujian,Mainland China,26.0789,117.9874,1/22/20,1
4,Gansu,Mainland China,36.0611,103.8343,1/22/20,0


In [7]:
raw_data_confirmed['Date'] = pd.to_datetime(raw_data_confirmed['Date'])
raw_data_deaths['Date'] = pd.to_datetime(raw_data_deaths['Date'])
raw_data_Recovered['Date'] = pd.to_datetime(raw_data_Recovered['Date'])


In [8]:
# Renaming the Values column
raw_data_confirmed.columns = raw_data_confirmed.columns.str.replace('value', 'Confirmed')
raw_data_deaths.columns = raw_data_deaths.columns.str.replace('value', 'Deaths')
raw_data_Recovered.columns = raw_data_Recovered.columns.str.replace('value', 'Recovered')

In [9]:
raw_data_confirmed.head()

,Province/State,Country/Region,Lat,Long,Date,Confirmed
0,Anhui,Mainland China,31.8257,117.2264,2020-01-22,1
1,Beijing,Mainland China,40.1824,116.4142,2020-01-22,14
2,Chongqing,Mainland China,30.0572,107.8740,2020-01-22,6
3,Fujian,Mainland China,26.0789,117.9874,2020-01-22,1
4,Gansu,Mainland China,36.0611,103.8343,2020-01-22,0


In [10]:
raw_data_Recovered.isnull().sum()

Province/State    4324
Country/Region       0
Lat                  0
Long                 0
Date                 0
Recovered            0
dtype: int64

In [11]:
#  NULL values

raw_data_confirmed['Province/State'].fillna(raw_data_confirmed['Country/Region'], inplace=True)
raw_data_deaths['Province/State'].fillna(raw_data_deaths['Country/Region'], inplace=True)
raw_data_Recovered['Province/State'].fillna(raw_data_Recovered['Country/Region'], inplace=True)

raw_data_confirmed.isnull().sum()
raw_data_deaths.isnull().sum()
raw_data_Recovered.isnull().sum()

Province/State    0
Country/Region    0
Lat               0
Long              0
Date              0
Recovered         0
dtype: int64

In [12]:
#joining 3 tables

# Confirmed with Deaths
full_join = raw_data_confirmed.merge(raw_data_deaths[['Province/State','Country/Region','Date','Deaths']], 
                                      how = 'left', 
                                      left_on = ['Province/State','Country/Region','Date'], 
                                      right_on = ['Province/State', 'Country/Region','Date'])

print("Shape of first join: ", full_join.shape)

# full join with Recovered
full_join = full_join.merge(raw_data_Recovered[['Province/State','Country/Region','Date','Recovered']], 
                                      how = 'left', 
                                      left_on = ['Province/State','Country/Region','Date'], 
                                      right_on = ['Province/State', 'Country/Region','Date'])

print("Shape of second join: ", full_join.shape)

full_join.head()

Shape of first join:  (10350, 7)
Shape of second join:  (10350, 8)


,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered
0,Anhui,Mainland China,31.8257,117.2264,2020-01-22,1,0,0
1,Beijing,Mainland China,40.1824,116.4142,2020-01-22,14,0,0
2,Chongqing,Mainland China,30.0572,107.8740,2020-01-22,6,0,0
3,Fujian,Mainland China,26.0789,117.9874,2020-01-22,1,0,0
4,Gansu,Mainland China,36.0611,103.8343,2020-01-22,0,0,0


In [13]:
full_join.isnull().sum()

Province/State    0
Country/Region    0
Lat               0
Long              0
Date              0
Confirmed         0
Deaths            0
Recovered         0
dtype: int64

In [14]:
# Adding Month and Year as a new Column
full_join['Month-Year'] = full_join['Date'].dt.strftime('%b-%Y')

In [15]:
full_join.head()

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered,Month-Year
0,Anhui,Mainland China,31.8257,117.2264,2020-01-22,1,0,0,Jan-2020
1,Beijing,Mainland China,40.1824,116.4142,2020-01-22,14,0,0,Jan-2020
2,Chongqing,Mainland China,30.0572,107.8740,2020-01-22,6,0,0,Jan-2020
3,Fujian,Mainland China,26.0789,117.9874,2020-01-22,1,0,0,Jan-2020
4,Gansu,Mainland China,36.0611,103.8343,2020-01-22,0,0,0,Jan-2020


In [16]:
#creating a new df    
test = full_join[full_join['Province/State'] == 'Anhui']

#creating a new df    
full_join2 = test.copy()

#creating a new date columns - 1
full_join2['Date - 1'] = full_join2['Date'] + pd.Timedelta(days=1)
full_join2.rename(columns={'Confirmed': 'Confirmed - 1', 'Deaths': 'Deaths - 1', 'Recovered': 'Recovered - 1',
                          'Date': 'Date Minus 1'}, inplace=True)

#Joing on the 2 DFs
full_join3 = test.merge(full_join2[['Province/State', 'Country/Region','Confirmed - 1', 'Deaths - 1', 
                            'Recovered - 1', 'Date - 1', 'Date Minus 1']], how = 'outer',
                             left_on = ['Province/State','Country/Region','Date'], 
                             right_on = ['Province/State', 'Country/Region','Date - 1'])

# Additional Calculations
full_join3['Confirmed Daily'] = full_join3['Confirmed'] - full_join3['Confirmed - 1']


test.head()
full_join2.head()
full_join3.head()

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered,Month-Year,Confirmed - 1,Deaths - 1,Recovered - 1,Date - 1,Date Minus 1,Confirmed Daily
0,Anhui,Mainland China,31.8257,117.2264,2020-01-22,1.0,0.0,0.0,Jan-2020,NaN,NaN,NaN,NaT,NaT,NaN
1,Anhui,Mainland China,31.8257,117.2264,2020-01-23,9.0,0.0,0.0,Jan-2020,1.0,0.0,0.0,2020-01-23,2020-01-22,8.0
2,Anhui,Mainland China,31.8257,117.2264,2020-01-24,15.0,0.0,0.0,Jan-2020,9.0,0.0,0.0,2020-01-24,2020-01-23,6.0
3,Anhui,Mainland China,31.8257,117.2264,2020-01-25,39.0,0.0,0.0,Jan-2020,15.0,0.0,0.0,2020-01-25,2020-01-24,24.0
4,Anhui,Mainland China,31.8257,117.2264,2020-01-26,60.0,0.0,0.0,Jan-2020,39.0,0.0,0.0,2020-01-26,2020-01-25,21.0


In [17]:
test.head()

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered,Month-Year
0,Anhui,Mainland China,31.8257,117.2264,2020-01-22,1,0,0,Jan-2020
225,Anhui,Mainland China,31.8257,117.2264,2020-01-23,9,0,0,Jan-2020
450,Anhui,Mainland China,31.8257,117.2264,2020-01-24,15,0,0,Jan-2020
675,Anhui,Mainland China,31.8257,117.2264,2020-01-25,39,0,0,Jan-2020
900,Anhui,Mainland China,31.8257,117.2264,2020-01-26,60,0,0,Jan-2020


In [18]:
full_join2.head()

,Province/State,Country/Region,Lat,Long,Date Minus 1,Confirmed - 1,Deaths - 1,Recovered - 1,Month-Year,Date - 1
0,Anhui,Mainland China,31.8257,117.2264,2020-01-22,1,0,0,Jan-2020,2020-01-23
225,Anhui,Mainland China,31.8257,117.2264,2020-01-23,9,0,0,Jan-2020,2020-01-24
450,Anhui,Mainland China,31.8257,117.2264,2020-01-24,15,0,0,Jan-2020,2020-01-25
675,Anhui,Mainland China,31.8257,117.2264,2020-01-25,39,0,0,Jan-2020,2020-01-26
900,Anhui,Mainland China,31.8257,117.2264,2020-01-26,60,0,0,Jan-2020,2020-01-27


In [19]:
## Applying it on all dataset

#creating a new df    
full_join2 = full_join.copy()

#creating a new date columns - 1
full_join2['Date - 1'] = full_join2['Date'] + pd.Timedelta(days=1)
full_join2.rename(columns={'Confirmed': 'Confirmed - 1', 'Deaths': 'Deaths - 1', 'Recovered': 'Recovered - 1',
                          'Date': 'Date Minus 1'}, inplace=True)

#Joing on the 2 DFs
full_join3 = full_join.merge(full_join2[['Province/State', 'Country/Region','Confirmed - 1', 'Deaths - 1', 
                            'Recovered - 1', 'Date - 1', 'Date Minus 1']], how = 'left',
                             left_on = ['Province/State','Country/Region','Date'], 
                             right_on = ['Province/State', 'Country/Region','Date - 1'])

#minus_onedf.rename(columns={'Confirmed': 'Confirmed - 1', 'Deaths': 'Deaths - 1', 'Recovered': 'Recovered - 1'}, inplace=True)

full_join3.head()

# Additional Calculations
full_join3['Confirmed Daily'] = full_join3['Confirmed'] - full_join3['Confirmed - 1']
full_join3['Deaths Daily'] = full_join3['Deaths'] - full_join3['Deaths - 1']
full_join3['Recovered Daily'] = full_join3['Recovered'] - full_join3['Recovered - 1']

print(full_join3.shape)

(10350, 17)


In [20]:
full_join3.head()

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered,Month-Year,Confirmed - 1,Deaths - 1,Recovered - 1,Date - 1,Date Minus 1,Confirmed Daily,Deaths Daily,Recovered Daily
0,Anhui,Mainland China,31.8257,117.2264,2020-01-22,1,0,0,Jan-2020,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN
1,Beijing,Mainland China,40.1824,116.4142,2020-01-22,14,0,0,Jan-2020,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN
2,Chongqing,Mainland China,30.0572,107.8740,2020-01-22,6,0,0,Jan-2020,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN
3,Fujian,Mainland China,26.0789,117.9874,2020-01-22,1,0,0,Jan-2020,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN
4,Gansu,Mainland China,36.0611,103.8343,2020-01-22,0,0,0,Jan-2020,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN


In [21]:
# Additing manually the numbers for first day

full_join3['Confirmed Daily'].loc[full_join3['Date'] == '2020-01-22'] = full_join3['Confirmed']
full_join3['Deaths Daily'].loc[full_join3['Date'] == '2020-01-22'] = full_join3['Deaths']
full_join3['Recovered Daily'].loc[full_join3['Date'] == '2020-01-22'] = full_join3['Recovered']

C:\Users\Admin\AppData\Local\Temp\ipykernel_16288\2528994319.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  full_join3['Confirmed Daily'].loc[full_join3['Date'] == '2020-01-22'] = full_join3['Confirmed']
C:\Users\Admin\AppData\Local\Temp\ipykernel_16288\2528994319.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  full_join3['Deaths Daily'].loc[full_join3['Date'] == '2020-01-22'] = full_join3['Deaths']
C:\Users\Admin\AppData\Local\Temp\ipykernel_16288\2528994319.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.py

In [22]:
# deleting columns
del full_join3['Confirmed - 1']
del full_join3['Deaths - 1']
del full_join3['Recovered - 1']
del full_join3['Date - 1']
del full_join3['Date Minus 1']

In [23]:
full_join3

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered,Month-Year,Confirmed Daily,Deaths Daily,Recovered Daily
0,Anhui,Mainland China,31.8257,117.2264,2020-01-22,1,0,0,Jan-2020,1.0,0.0,0.0
1,Beijing,Mainland China,40.1824,116.4142,2020-01-22,14,0,0,Jan-2020,14.0,0.0,0.0
2,Chongqing,Mainland China,30.0572,107.8740,2020-01-22,6,0,0,Jan-2020,6.0,0.0,0.0
3,Fujian,Mainland China,26.0789,117.9874,2020-01-22,1,0,0,Jan-2020,1.0,0.0,0.0
4,Gansu,Mainland China,36.0611,103.8343,2020-01-22,0,0,0,Jan-2020,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
10345,"Pierce County, WA",US,47.0676,-122.1295,2020-03-07,1,0,0,Mar-2020,1.0,0.0,0.0
10346,"Plymouth County, MA",US,42.1615,-70.7928,2020-03-07,1,0,0,Mar-2020,0.0,0.0,0.0
10347,"Santa Cruz County, CA",US,36.9741,-122.0308,2020-03-07,1,0,0,Mar-2020,1.0,0.0,0.0
10348,"Tulsa County, OK",US,36.1593,-95.9410,2020-03-07,1,0,0,Mar-2020,1.0,0.0,0.0


In [24]:
# Creating additional slicer for easy of use

full_join3['Hubei Vs Rest of the World'] = 'Rest of the World'
full_join3['Hubei Vs Rest of the World'].loc[full_join3['Province/State'] == 'Hubei'] = 'Hubei - Virus birth'

#full_join3[full_join3['Province/State'] == 'Hubei']

C:\Users\Admin\AppData\Local\Temp\ipykernel_16288\1742397093.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  full_join3['Hubei Vs Rest of the World'].loc[full_join3['Province/State'] == 'Hubei'] = 'Hubei - Virus birth'


In [25]:
full_join3.head()

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered,Month-Year,Confirmed Daily,Deaths Daily,Recovered Daily,Hubei Vs Rest of the World
0,Anhui,Mainland China,31.8257,117.2264,2020-01-22,1,0,0,Jan-2020,1.0,0.0,0.0,Rest of the World
1,Beijing,Mainland China,40.1824,116.4142,2020-01-22,14,0,0,Jan-2020,14.0,0.0,0.0,Rest of the World
2,Chongqing,Mainland China,30.0572,107.8740,2020-01-22,6,0,0,Jan-2020,6.0,0.0,0.0,Rest of the World
3,Fujian,Mainland China,26.0789,117.9874,2020-01-22,1,0,0,Jan-2020,1.0,0.0,0.0,Rest of the World
4,Gansu,Mainland China,36.0611,103.8343,2020-01-22,0,0,0,Jan-2020,0.0,0.0,0.0,Rest of the World


In [35]:
# Exporting the data

# Setting my path
path = "C:\\Users\\Admin\\Desktop\\powerbi\\covid-19"

# Changing my CWD
os.chdir(path)

full_join3.to_csv('CoronaVirus PowerBI Raw', sep='\t')

In [36]:
# puting unique values in list
dates = full_join3['Date'].unique()

#creating a df with unique
dates = pd.DataFrame(dates, columns=['Date'])

# Ordering df
dates = dates.sort_values(by=['Date'])

# Creating an ordered list now
dates = full_join3['Date'].unique()

print(full_join3.shape)

try:
    del concat_data
except:
    print()
    
try:
    del final_concat_data
except:
    print()
    

for i in dates:
    new_data = full_join3[full_join3['Date'] == i]
    new_data['Cumulative Date'] = i
    print(i)
    
    try:     
        concat_data = pd.concat([concat_data, new_data], ignore_index = True)
        concat_data['Cumulative Date 2'] = i
        print(concat_data['Date'].unique())
        
        try:
            final_concat_data = pd.concat([final_concat_data, concat_data], ignore_index = True)
        except:
            final_concat_data = concat_data

    except:
        concat_data = new_data
        
print(final_concat_data.shape)

(10350, 13)


2020-01-22T00:00:00.000000000
2020-01-23T00:00:00.000000000
['2020-01-22T00:00:00.000000000' '2020-01-23T00:00:00.000000000']
2020-01-24T00:00:00.000000000
['2020-01-22T00:00:00.000000000' '2020-01-23T00:00:00.000000000'
 '2020-01-24T00:00:00.000000000']
2020-01-25T00:00:00.000000000
['2020-01-22T00:00:00.000000000' '2020-01-23T00:00:00.000000000'
 '2020-01-24T00:00:00.000000000' '2020-01-25T00:00:00.000000000']
2020-01-26T00:00:00.000000000
['2020-01-22T00:00:00.000000000' '2020-01-23T00:00:00.000000000'
 '2020-01-24T00:00:00.000000000' '2020-01-25T00:00:00.000000000'
 '2020-01-26T00:00:00.000000000']
2020-01-27T00:00:00.000000000
['2020-01-22T00:00:00.000000000' '2020-01-23T00:00:00.000000000'
 '2020-01-24T00:00:00.000000000' '2020-01-25T00:00:00.000000000'
 '2020-01-26T00:00:00.000000000' '2020-01-27T00:00:00.000000000']
2020-01-28T00:00:00.000000000
['2020-01-22T00:00:00.000000000' '2020-01-23T00:00:00.000000000'
 '2020-01-24T00:00:00.000000000' '2020-01-25T00:00:00.0

C:\Users\Admin\AppData\Local\Temp\ipykernel_16288\3429718920.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_data['Cumulative Date'] = i
C:\Users\Admin\AppData\Local\Temp\ipykernel_16288\3429718920.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_data['Cumulative Date'] = i
C:\Users\Admin\AppData\Local\Temp\ipykernel_16288\3429718920.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See

['2020-01-22T00:00:00.000000000' '2020-01-23T00:00:00.000000000'
 '2020-01-24T00:00:00.000000000' '2020-01-25T00:00:00.000000000'
 '2020-01-26T00:00:00.000000000' '2020-01-27T00:00:00.000000000'
 '2020-01-28T00:00:00.000000000' '2020-01-29T00:00:00.000000000'
 '2020-01-30T00:00:00.000000000' '2020-01-31T00:00:00.000000000'
 '2020-02-01T00:00:00.000000000' '2020-02-02T00:00:00.000000000'
 '2020-02-03T00:00:00.000000000']
2020-02-04T00:00:00.000000000
['2020-01-22T00:00:00.000000000' '2020-01-23T00:00:00.000000000'
 '2020-01-24T00:00:00.000000000' '2020-01-25T00:00:00.000000000'
 '2020-01-26T00:00:00.000000000' '2020-01-27T00:00:00.000000000'
 '2020-01-28T00:00:00.000000000' '2020-01-29T00:00:00.000000000'
 '2020-01-30T00:00:00.000000000' '2020-01-31T00:00:00.000000000'
 '2020-02-01T00:00:00.000000000' '2020-02-02T00:00:00.000000000'
 '2020-02-03T00:00:00.000000000' '2020-02-04T00:00:00.000000000']
2020-02-05T00:00:00.000000000
['2020-01-22T00:00:00.000000000' '2020-01-23T00:00:00.000000

C:\Users\Admin\AppData\Local\Temp\ipykernel_16288\3429718920.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_data['Cumulative Date'] = i
C:\Users\Admin\AppData\Local\Temp\ipykernel_16288\3429718920.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_data['Cumulative Date'] = i
C:\Users\Admin\AppData\Local\Temp\ipykernel_16288\3429718920.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See

['2020-01-22T00:00:00.000000000' '2020-01-23T00:00:00.000000000'
 '2020-01-24T00:00:00.000000000' '2020-01-25T00:00:00.000000000'
 '2020-01-26T00:00:00.000000000' '2020-01-27T00:00:00.000000000'
 '2020-01-28T00:00:00.000000000' '2020-01-29T00:00:00.000000000'
 '2020-01-30T00:00:00.000000000' '2020-01-31T00:00:00.000000000'
 '2020-02-01T00:00:00.000000000' '2020-02-02T00:00:00.000000000'
 '2020-02-03T00:00:00.000000000' '2020-02-04T00:00:00.000000000'
 '2020-02-05T00:00:00.000000000' '2020-02-06T00:00:00.000000000'
 '2020-02-07T00:00:00.000000000' '2020-02-08T00:00:00.000000000'
 '2020-02-09T00:00:00.000000000' '2020-02-10T00:00:00.000000000'
 '2020-02-11T00:00:00.000000000' '2020-02-12T00:00:00.000000000']
2020-02-13T00:00:00.000000000
['2020-01-22T00:00:00.000000000' '2020-01-23T00:00:00.000000000'
 '2020-01-24T00:00:00.000000000' '2020-01-25T00:00:00.000000000'
 '2020-01-26T00:00:00.000000000' '2020-01-27T00:00:00.000000000'
 '2020-01-28T00:00:00.000000000' '2020-01-29T00:00:00.00000

C:\Users\Admin\AppData\Local\Temp\ipykernel_16288\3429718920.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_data['Cumulative Date'] = i
C:\Users\Admin\AppData\Local\Temp\ipykernel_16288\3429718920.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_data['Cumulative Date'] = i
C:\Users\Admin\AppData\Local\Temp\ipykernel_16288\3429718920.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See

['2020-01-22T00:00:00.000000000' '2020-01-23T00:00:00.000000000'
 '2020-01-24T00:00:00.000000000' '2020-01-25T00:00:00.000000000'
 '2020-01-26T00:00:00.000000000' '2020-01-27T00:00:00.000000000'
 '2020-01-28T00:00:00.000000000' '2020-01-29T00:00:00.000000000'
 '2020-01-30T00:00:00.000000000' '2020-01-31T00:00:00.000000000'
 '2020-02-01T00:00:00.000000000' '2020-02-02T00:00:00.000000000'
 '2020-02-03T00:00:00.000000000' '2020-02-04T00:00:00.000000000'
 '2020-02-05T00:00:00.000000000' '2020-02-06T00:00:00.000000000'
 '2020-02-07T00:00:00.000000000' '2020-02-08T00:00:00.000000000'
 '2020-02-09T00:00:00.000000000' '2020-02-10T00:00:00.000000000'
 '2020-02-11T00:00:00.000000000' '2020-02-12T00:00:00.000000000'
 '2020-02-13T00:00:00.000000000' '2020-02-14T00:00:00.000000000'
 '2020-02-15T00:00:00.000000000' '2020-02-16T00:00:00.000000000'
 '2020-02-17T00:00:00.000000000' '2020-02-18T00:00:00.000000000'
 '2020-02-19T00:00:00.000000000' '2020-02-20T00:00:00.000000000'
 '2020-02-21T00:00:00.000

C:\Users\Admin\AppData\Local\Temp\ipykernel_16288\3429718920.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_data['Cumulative Date'] = i
C:\Users\Admin\AppData\Local\Temp\ipykernel_16288\3429718920.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_data['Cumulative Date'] = i
C:\Users\Admin\AppData\Local\Temp\ipykernel_16288\3429718920.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See

2020-03-01T00:00:00.000000000
['2020-01-22T00:00:00.000000000' '2020-01-23T00:00:00.000000000'
 '2020-01-24T00:00:00.000000000' '2020-01-25T00:00:00.000000000'
 '2020-01-26T00:00:00.000000000' '2020-01-27T00:00:00.000000000'
 '2020-01-28T00:00:00.000000000' '2020-01-29T00:00:00.000000000'
 '2020-01-30T00:00:00.000000000' '2020-01-31T00:00:00.000000000'
 '2020-02-01T00:00:00.000000000' '2020-02-02T00:00:00.000000000'
 '2020-02-03T00:00:00.000000000' '2020-02-04T00:00:00.000000000'
 '2020-02-05T00:00:00.000000000' '2020-02-06T00:00:00.000000000'
 '2020-02-07T00:00:00.000000000' '2020-02-08T00:00:00.000000000'
 '2020-02-09T00:00:00.000000000' '2020-02-10T00:00:00.000000000'
 '2020-02-11T00:00:00.000000000' '2020-02-12T00:00:00.000000000'
 '2020-02-13T00:00:00.000000000' '2020-02-14T00:00:00.000000000'
 '2020-02-15T00:00:00.000000000' '2020-02-16T00:00:00.000000000'
 '2020-02-17T00:00:00.000000000' '2020-02-18T00:00:00.000000000'
 '2020-02-19T00:00:00.000000000' '2020-02-20T00:00:00.000000

C:\Users\Admin\AppData\Local\Temp\ipykernel_16288\3429718920.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_data['Cumulative Date'] = i
C:\Users\Admin\AppData\Local\Temp\ipykernel_16288\3429718920.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_data['Cumulative Date'] = i
C:\Users\Admin\AppData\Local\Temp\ipykernel_16288\3429718920.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See

In [37]:
final_concat_data.to_csv('CoronaVirus PowerBI Raw - Cumulative Test', sep='\t')